In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

In [3]:
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")
llm = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=500)

In [4]:
# Carregar o PDF
pdf_link = '../../pdfs/weight-loss.pdf'

loader = PyPDFLoader(pdf_link, extract_images=False)

pages = loader.load_and_split()

len(pages)

8

In [5]:
# Splitter
child_splitter = RecursiveCharacterTextSplitter(
  chunk_size=200,
)

parent_splitter = RecursiveCharacterTextSplitter(
  chunk_size=4000,
  chunk_overlap=200,
  length_function=len,
  add_start_index=True,
)

In [6]:
# Storages
store = InMemoryStore()
vectorstore = Chroma(
  embedding_function=embedding,
  persist_directory='childVectorDB',
)

In [7]:
parent_document_retriever = ParentDocumentRetriever(
  vectorstore=vectorstore,
  docstore=store,
  child_splitter=child_splitter,
  parent_splitter=parent_splitter
)

parent_document_retriever.add_documents(pages, ids=None)

In [8]:
TEMPLATE = """
  Você é um especialista em nutrição e saúde. Sua tarefa é responder perguntas sobre o documento fornecido.
  Se a pergunta do usuário não está relacionada ao documento, responda que não sabe.

  Documento:
  {context}

  Pergunta:
  {question}
"""

rag_prompt = ChatPromptTemplate.from_template(
  TEMPLATE,
)


In [9]:
setup_retrieval = RunnableParallel({
  'question': RunnablePassthrough(),
  'context': parent_document_retriever
})

output_parser = StrOutputParser()

In [10]:
parent_chain_retrieval = setup_retrieval | rag_prompt | llm | output_parser

In [11]:
parent_chain_retrieval.invoke('Como perder peso de maneira rápida e saudável?')

'Para perder peso de maneira rápida e saudável, é importante consumir menos calorias do que você queima. Aumentar a atividade física enquanto limita a ingestão de calorias pode acelerar a perda de peso. Além disso, é essencial manter um registro escrito da alimentação e da atividade física, pesar-se uma vez por semana no mesmo horário e com a mesma quantidade de roupa, e seguir outras diretrizes como comer devagar, priorizar alimentos ricos em proteínas, ler rótulos de alimentos, evitar alimentos ricos em gordura e açúcar, e controlar as porções. É fundamental discutir níveis adequados de calorias e tamanhos de porções com um nutricionista para um plano personalizado.'